In [ ]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

## Arnoldi 

This is a copy of the algorithm defined and tested in the notebook `algo2_testing`. It's an implementation of the Algorithm 2 from the paper. It's needed in this notebook since this function is called by the `algo4` function. It's implemented to return exactly what's needed in the `algo4` function.

Everything will be reorganized in the main.py file once everything is working.

In [ ]:
def Arnoldi(A,v0,m):
    v = v0
    beta = norm(v)
    v = v/beta
    H = sp.sparse.lil_matrix((m+1,m)) 
    V = sp.sparse.lil_matrix((A.shape[0],m+1))
    V[:,0] = v # each column of V is a vector v

    for j in range(m):
        w = A @ v  
        for i in range(j):
            tmp = v.T @ w # tmp is a 1x1 matrix, so it's O(1) in memory
            H[i,j] = tmp[0,0] 
            w = w - H[i,j]*v 
            
        H[j+1,j] = norm(w)

        if H[j+1,j] == 0:
            # print("Arnoldi breakdown")
            m = j
            v = 0
            break
        else:
            if j < m-1:
                v = w/H[j+1,j]
                V[:,j+1] = v

    return V, H, v, beta, j  

# Algorithm 4 testing

This algorithm is based on the "Algorithm 4" of the paper, the pseudocode provided by the authors is the following 

![](https://i.imgur.com/H92fru7.png)

Line 14 is particularly tricky to understand, not working for now. Need to figure out how to solve that linear system. My idea was to do something like that

![](https://i.imgur.com/uBCDYUa.jpeg)

And use the `sp.sparse.linalg.spsolve` function to solve the linear system as $Ax=0$ where $A$ is $[\bar H_m^i ~ |  ~ z]$ but it returns an array of zeros. So the idea it's wrong

In [ ]:
def Algo4(Pt, v, m, a: list, tau, maxit: int, x):
    
    # I'm using a non declared variable n here , it's declared in the next cell when I call this function. This will be fixed later in the main.py file

    iter = 1
    mv = 0
    I = sp.sparse.eye(n, n, format='lil')
    r = sp.sparse.lil_matrix((n,1))
    res = np.zeros(len(a)) 

    # I'm defining 3 canonical vectors of different sizes. It's probably stupid, will be fixed once the algorithm actually works

    H_e1 = np.zeros((m+1,1)) # canonical basis vector of size H.shape[0]
    H_e1[0] = 1

    V_e1 = np.zeros((n,1)) # canonical basis vector of size V.shape[0]
    V_e1[0] = 1

    s_e1 = np.zeros((len(a),1)) # canonical basis vector of size s.shape[0]
    s_e1[0] = 1

    def find_k(res): # function to find the index of the largest element in res
        k = 0
        for i in range(len(a)):
            if res[i] == max(res):
                k = i
                break
        return k

    def compute_gamma(res, a, k): # function to compute gamma
        gamma = np.zeros(len(a))
        for i in range(len(a)):
            if i != k:
                gamma[i] = (res[i]*a[k])/(res[k]*a[i])
            else:
                gamma[i] = 0
        return gamma

    # compute the residual vector
    for i in range(len(a)):
        r = ((1-a[i])/a[i])*v - ((1/a[i])*I - Pt) @ x
        res[i] = a[i]*norm(r)

    while max(res) >= tau and iter <= maxit:
        k = find_k(res)
        gamma = compute_gamma(res, a, k)
        V, H, v, beta, j = Arnoldi((1/a[k])*I - Pt, r, m)

        mv = mv + j

        # compute y as the minimizer of || beta*e1 - Hy ||_2 using the least squares method
        y = sp.sparse.linalg.lsqr(H, beta*H_e1)[0]

        # reshape y to be a column vector
        y = y.reshape(y.shape[0],1)

        # update x  
        x += V[:,0:y.shape[0]] @ y

        # compute the residual vector
        res[k] = a[k]*np.linalg.norm(beta*V_e1 - V[:,0:y.shape[0]] @ y)
        
        # for i in range(len(a)) but not k
        for i in range(len(a)):
            if i != k and res[i] >= tau:
                # Compute H as described in the paper
                H = H + ((1-a[i])/a[i] - (1-a[k])/a[k])*sp.sparse.eye(H.shape[0], H.shape[1], format='lil') 

                z = beta*H_e1 - H @ y # define z as in the paper (page 9)
                A_tmp = sp.sparse.hstack([H, z]) # stack H and z, as in the paper, to solve the linear system (?)
                A_tmp = A_tmp.tocsc() # Convert A to CSC format for sparse solver

                # What should I put here? What does it mean in the paper the line 14 of the pseudocode?
                result = sp.sparse.linalg.spsolve(A_tmp, np.zeros(A_tmp.shape[0])) # if I solve this, I get a vector of zeros.
                print(result)
                
                # I don't know if the code below is correct since I don't get how to solve the linear system above, so I'm unsure about what y and gamma should be. For now it's commented out.

                # # update x
                # x += V[:,0:y.shape[0]] @ y
                # # update the residual vector
                # res[i] = (a[i]/a[k])*gamma[k]*res[k] 

        iter = iter + 1

    return x, iter, mv

Basic test case with random numbers to test the algorithm.

In [ ]:
n = 100
m = 110
maxit = 100
tau = 1e-6
a = [0.85, 0.9, 0.95, 0.99]

x = sp.sparse.lil_matrix((n,1))
x[0,0] = 1

# generate a random graph
G = nx.gnp_random_graph(n, 0.1, seed=1, directed=True)

P = sp.sparse.lil_matrix((n,n))
for i in G.nodes():
    for j in G[i]: #G[i] is the list of nodes connected to i, it's neighbors
        P[i-1,j-1] = 1/len(G[i])

# generate a probability vector, with all the entries as 1/n
v = sp.sparse.lil_matrix((n,1))
for i in range(n):
    v[i] = 1/n

# dangling nodes vector
d = sp.sparse.lil_matrix((n,1))
for i in range(n):
    if P[i].sum() == 0:
        d[i] = 1

# compute the transition matrix
Pt = P + v @ (d.T)

# run the algorithm
Algo4(Pt, v, m, a, tau, maxit, x)